# A360 Red Sequence DESI Spec-z vs Photo-z 

Prakruth Adari, Tianqing Zhang, Anja von der Linden\
LSST Science Piplines version: Weekly 2025_20\
Container Size: large

A quick look into the photo-z estimates and DESI spec-zs 

In [ ]:
!eups list -s | grep lsst_distrib

In [ ]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.spatial import KDTree
import scipy.interpolate as interpolate
from scipy.optimize import curve_fit
import scipy.integrate as integrate
import scipy.stats as stats
import healpy as hp
import pandas as pd
from matplotlib import cm
from astropy.visualization import make_lupton_rgb
from matplotlib.legend_handler import HandlerLine2D
from astropy.table import Table, join, vstack
from astropy.io import fits
from astropy.coordinates import SkyCoord
# %matplotlib widget

In [ ]:
# Familiar stack packages
from lsst.daf.butler import Butler
from lsst.geom import Box2I, Box2D, Point2I, Point2D, Extent2I, Extent2D
# from lsst.afw.image import Exposure, Image, PARENT
import lsst.sphgeom

# These may be less familiar objects dealing with multi-band data products
from lsst.afw.image import MultibandExposure, MultibandImage

In [ ]:
# import lsst.afw.display as afwDisplay
import os, sys

In [ ]:

%matplotlib inline
%config InlineBackend.figure_format='retina'


In [ ]:
hdir = os.getenv('HOME')
ddir = '/home/a/adari/DATA'

arcsec = 1/60**2

rng = np.random.default_rng()

In [ ]:
cluster_coords = np.array([37.86501659859067, 6.982204815599694])

### Loading in A360 Photo-z data 

Load in the photo-z table and label red sequence members

In [ ]:
table = pd.read_parquet('/sdf/data/rubin/shared/cluster_commissioning/pz_fzb/a360_with_flexzboost_tpz_dp1_freeze.pq') 
table = Table.from_pandas(table)

In [ ]:
color_ri = table['r_mag'] - table['i_mag']
color_gi = table['g_mag'] - table['i_mag']

In [ ]:
gi_rs = 1.6 - (0.07) * (table['i_mag']-21)
gi_rs_hi = gi_rs + 0.22
gi_rs_low = gi_rs - 0.12

gi_rs_lbl = ((color_gi > gi_rs_low) * (color_gi < gi_rs_hi) * (table['i_mag'] < 24.5))

ri_rs = 0.5 - (0.02) * (table['i_mag']-21)
ri_rs_hi = ri_rs + 0.05
ri_rs_low = ri_rs - 0.07

ri_rs_lbl = ((color_ri > ri_rs_low) * (color_ri < ri_rs_hi) * (table['i_mag'] < 24.5))

rigi_rs = ri_rs_lbl * gi_rs_lbl

In [ ]:
color_ri = table['r_mag'] - table['i_mag']
# c_rs_hi = 0.6 - (0.1/5.) * (table['r_mag']-19)
# c_rs_low = 0.4 - (0.1/5.)* (table['r_mag']-19)

c_rs_hi = 0.65 - (0.1/5.) * (table['r_mag']-19)
c_rs_low = 0.42 - (0.1/5.)* (table['r_mag']-19)

ri_rs = ((color_ri > c_rs_low) * (color_ri < c_rs_hi) * (table['r_mag'] < 23))

In [ ]:
table['rigi_rs'] = rigi_rs
table['ri_rs'] = ri_rs

### Load DESI Spec-z Data

The LRG and BGS redshift catalogs are stored in `/sdf/data/rubin/shared/cluster_commissioning/desi`

In [ ]:
# os.listdir('/sdf/data/rubin/shared/cluster_commissioning/desi/')

In [ ]:
with fits.open('/sdf/data/rubin/shared/cluster_commissioning/desi/LRG_SGC_clustering.dat.fits') as hdul:
    data = hdul[1].data
    LRG = Table(data)

with fits.open('/sdf/data/rubin/shared/cluster_commissioning/desi/BGS_ANY_SGC_clustering.dat.fits') as hdul:
    data = hdul[1].data
    BGS = Table(data)

In [ ]:
desi_data = vstack([LRG, BGS])

In [ ]:
# Restrict the DESI data to only those within 1 degree of the BCG

c1 = SkyCoord(desi_data['RA']*u.deg, desi_data['DEC']*u.deg)
c2 = SkyCoord(cluster_coords[0]*u.deg, cluster_coords[1]*u.deg)
sep = c1.separation(c2)

near_filt = (sep < 1*u.deg)
near_desi = desi_data[near_filt]
print(len(near_desi))

In [ ]:
# Restricting each subsample for plotting
lrgc = SkyCoord(LRG['RA']*u.deg, LRG['DEC']*u.deg)
bgsc = SkyCoord(BGS['RA']*u.deg, BGS['DEC']*u.deg)
c2 = SkyCoord(cluster_coords[0]*u.deg, cluster_coords[1]*u.deg)
sep_lrg = lrgc.separation(c2)
sep_bgs = bgsc.separation(c2)

LRG_filt = sep_lrg < 1*u.deg
BGS_filt = sep_bgs < 1*u.deg
b1 = BGS[BGS_filt]
l1 = LRG[LRG_filt]
cmap = cm.coolwarm
print(np.sum(LRG_filt), np.sum(BGS_filt))

In [ ]:
plt.scatter(b1['RA'], b1['DEC'], 8, c=b1['Z'], cmap=cmap, label='BGS', marker='s', vmin=0.2, vmax=1.1)
plt.scatter(l1['RA'], l1['DEC'], 8, c=l1['Z'], cmap=cmap, label='LRG', marker='x', vmin=0.2, vmax=1.1)
plt.plot(cluster_coords[0], cluster_coords[1], 'k*')
plt.plot(table['coord_ra'], table['coord_dec'], ',', alpha=0.05)
plt.colorbar()
plt.legend(frameon=False)

### Match Catalogs

We can use [`friendly`](https://github.com/LSSTDESC/friendly/tree/FoF-friendly) to perform the matching for us but for clarity we will use a simple KDTree matcher.

In [ ]:
search_radius = 0.5 * arcsec

In [ ]:
desi_coords = np.vstack((near_desi['RA'], near_desi['DEC'])).T
desi_tree = KDTree(desi_coords)

When referring to the photometry DP1 data, I will refer to it as "CC" which stands for "Cluster Commissioning"


In [ ]:
cc_coords = np.vstack((table['coord_ra'], table['coord_dec'])).T
cc_tree = KDTree(cc_coords)

In [ ]:
# Distance of CC object to nearest 3 DESI matches
# NDX of nearest 3 DESI matches
# desi_dist, desi_ndx = desi_tree.query(cc_coords, k=3)

In [ ]:
# Distance of DESI object to nearest 3 CC matches
# NDX of nearest 3 CC matches
cc_dists, cc_ndx = cc_tree.query(desi_coords, k=3)

In [ ]:
# Does any DESI object have 2 CC objects within the search radius?
# If this is > 0 we have some bad matches!
print("Possible bad matches? Should be 0:", np.sum(cc_dist[:,1] < search_radius))
print("Possible good mathces", np.sum(cc_dist[:,0] < search_radius))

In [ ]:
match_ndx = [] # CC index
match_z = [] # DESI redshift
desi_ndx = np.where(cc_dists[:,0] < search_radius)[0] # DESI ndx of matched

# This could be done more elegantly but leaving it as the straightforward way for now

for i in range(len(desi_coords)):
    if not cc_dists[i, 0] < search_radius:
        continue

    match_ndx.append(cc_ndx[i,0])
    match_z.append(near_desi['Z'][i])

In [ ]:
# Add zs to table
insert_zs = np.ones(len(table)) * -99
insert_zs[match_ndx] = match_z

table['desi_z'] = insert_zs

### Matched Comparison

In [ ]:
matched = table[table['desi_z'] > 0]

In [ ]:
# cluster_membs = np.sqrt((match_cc['desi_z'] - .22)**2) < .01
rs_cc = matched[matched['rigi_rs']]

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12,6))

ax[0].plot(matched['desi_z'], matched['fzb_z_median'], '.')
ax[0].plot([0,10], [0,10], ls='--', color='k', alpha=.2)
ax[0].plot(rs_cc['desi_z'], rs_cc['fzb_z_median'], '.', color='r', label='Red Sequence')
ax[0].axvline(0.22, ls='--', color=cmap(0.95), alpha=0.3)
# ax[0].set_xlabel("Spectroscopic Redshift")
ax[0].set_ylabel("FZB Median Redshift")
# ax[0].legend(loc='lower right')
ax[0].set_xlim(0, 1.5)
ax[0].set_ylim(0, 1.5)

ax[1].plot(matched['desi_z'], matched['tpz_z_median'], '.')
ax[1].plot([0,10], [0,10], ls='--', color='k', alpha=.2)
ax[1].plot(rs_cc['desi_z'], rs_cc['tpz_z_median'], '.', color='r', label='Red Sequence')
ax[1].axvline(0.22, ls='--', color=cmap(0.95), alpha=0.3)
# ax[1].set_xlabel("Spectroscopic Redshift")
ax[1].set_ylabel("TPZ Median Redshift")
# ax[0].legend(loc='lower right')
ax[1].set_xlim(0, 1.5)
ax[1].set_ylim(0, 1.5)
ax[1].legend()
fig.supxlabel("Spectroscopic Redshift (MEDIAN)")
fig.suptitle("DESI Spec-Z PZ Comparison", y=.95)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12,6))

ax[0].plot(matched['desi_z'], matched['fzb_z_mean'], '.')
ax[0].plot([0,10], [0,10], ls='--', color='k', alpha=.2)
ax[0].plot(rs_cc['desi_z'], rs_cc['fzb_z_mean'], '.', color='r', label='Red Sequence')
ax[0].axvline(0.22, ls='--', color=cmap(0.95), alpha=0.3)
# ax[0].set_xlabel("Spectroscopic Redshift")
ax[0].set_ylabel("FZB Mean Redshift")
# ax[0].legend(loc='lower right')
ax[0].set_xlim(0, 1.5)
ax[0].set_ylim(0, 1.5)

ax[1].plot(matched['desi_z'], matched['tpz_z_mean'], '.')
ax[1].plot([0,10], [0,10], ls='--', color='k', alpha=.2)
ax[1].plot(rs_cc['desi_z'], rs_cc['tpz_z_mean'], '.', color='r', label='Red Sequence')
ax[1].axvline(0.22, ls='--', color=cmap(0.95), alpha=0.3)
# ax[1].set_xlabel("Spectroscopic Redshift")
ax[1].set_ylabel("TPZ Mean Redshift")
# ax[0].legend(loc='lower right')
ax[1].set_xlim(0, 1.5)
ax[1].set_ylim(0, 1.5)
ax[1].legend()
fig.supxlabel("Spectroscopic Redshift (MEAN)")
fig.suptitle("DESI Spec-Z PZ Comparison", y=.95)

In [ ]:
c1 = SkyCoord(table['coord_ra']*u.deg, table['coord_dec']*u.deg)
c2 = SkyCoord(cluster_coords[0]*u.deg, cluster_coords[1]*u.deg)
sep = c1.separation(c2)

near_filt = (sep < .1*u.deg)
near_bcg = table[near_filt]
near_bright = near_bcg[near_bcg['i_mag'] <= 24.5]

In [ ]:
redshift_dist = np.sqrt((matched['desi_z'] - 0.22)**2)
desi_cluster = matched[redshift_dist < 0.05]


In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10,5))

cmap = cm.coolwarm
# color_val = np.sqrt((match_cc['desi_z'] - 0.22)**2)

xs = np.linspace(15, 28, num=1001)

gi_rs_plot = 1.6 - (0.07) * (xs-21)
ri_rs_plot = 0.5 - (0.02) * (xs-21)

near_i = near_bcg['i_mag']
near_gi = near_bcg['g_mag'] - near_bcg['i_mag']
near_ri = near_bcg['r_mag'] - near_bcg['i_mag']

ax[0].plot(near_i, near_gi, '.', color=cmap(0.5), alpha=.25, label='ComCam')
ax[0].plot(matched['i_mag'], matched['g_mag'] - matched['i_mag'], '.', color=cmap(0.08), label='DESI Matches')
ax[0].plot(desi_cluster['i_mag'], desi_cluster['g_mag'] - desi_cluster['i_mag'], '.', color=cmap(0.98), label='DESI z=0.22')
ax[0].plot(xs, gi_rs_plot + 0.22, '--', color=cmap(0.22))
ax[0].plot(xs, gi_rs_plot - 0.12, '--', color=cmap(0.22))

ax[1].plot(near_i, near_ri, '.', color=cmap(0.5), alpha=.25, label='ComCam')
ax[1].plot(matched['i_mag'], matched['r_mag'] - matched['i_mag'],'.', color=cmap(0.08), label='DESI Matches')
ax[1].plot(desi_cluster['i_mag'], desi_cluster['r_mag'] - desi_cluster['i_mag'], '.', color=cmap(0.98), label='DESI z=0.22')
ax[1].plot(xs, ri_rs_plot + 0.05, '--', color=cmap(0.22))
ax[1].plot(xs, ri_rs_plot - 0.07, '--', color=cmap(0.22))



fig.supxlabel(r"$i$-mag")
ax[0].set_ylabel(r"$gi$")
ax[1].set_ylabel(r"$ri$")

for xx in ax:
    xx.set_xlim(15, 27)
    xx.axvline(24.5, ls='--', color='grey')
  
ax[0].set_ylim(-1, 4)
ax[1].set_ylim(-1, 2)
ax[1].legend(frameon=False, bbox_to_anchor=(1.1, .55))


In [ ]:
fig, ax = plt.subplots(1, figsize=(8,4))

ax.hist(matched['desi_z'][matched['rigi_rs']], range=(0, 3), bins=101, label='Matched Red Sequence', histtype='step')
ax.axvline(0.22, ls='--', color='k', alpha=.15, label='Cluster')
ax.set_xlabel('DESI spec-z')
ax.set_title(r"Near BCG + $i \leq 24.5$")
ax.legend()


In [ ]:
matched.write('/sdf/data/rubin/shared/cluster_commissioning/desi/a360_flexzboost_tpz_desimatch_dp1_freeze.fits', format='fits')